In [3]:
import pandas as pd

# Setup

In [4]:
training_data_fp = "./TrainingData.csv"
test_data_fp = "./TestData.csv"
submission_format_fp = "./SubmissionFormat.csv"

In [8]:
FEATURE_COLS = ['_id', 'Object_Description', 'Program_Description',
       'SubFund_Description', 'Job_Title_Description',
       'Facility_or_Department', 'Sub_Object_Description',
       'Location_Description', 'FTE', 'Function_Description', 'Position_Extra',
       'Text_4', 'Total', 'Text_2', 'Text_3', 'Fund_Description', 'Text_1']

In [9]:
TOP_LEVEL_LABELS = ['Function', 'Object_Type', 'Operating_Status', 'Position_Type', 'Pre_K', 'Reporting', 'Sharing', 'Student_Type', 'Use']

In [10]:
FLATTENED_LABELS = [

    'Function__Aides Compensation',
    'Function__Career & Academic Counseling',
    'Function__Communications',
    'Function__Curriculum Development',
    'Function__Data Processing & Information Services',
    'Function__Development & Fundraising',
    'Function__Enrichment',
    'Function__Extended Time & Tutoring',
    'Function__Facilities & Maintenance',
    'Function__Facilities Planning',
    'Function__Finance, Budget, Purchasing & Distribution',
    'Function__Food Services',
    'Function__Governance',
    'Function__Human Resources',
    'Function__Instructional Materials & Supplies',
    'Function__Insurance',
    'Function__Legal',
    'Function__Library & Media',
    'Function__NO_LABEL',
    'Function__Other Compensation',
    'Function__Other Non-Compensation',
    'Function__Parent & Community Relations',
    'Function__Physical Health & Services',
    'Function__Professional Development',
    'Function__Recruitment',
    'Function__Research & Accountability',
    'Function__School Administration',
    'Function__School Supervision',
    'Function__Security & Safety',
    'Function__Social & Emotional',
    'Function__Special Population Program Management & Support',
    'Function__Student Assignment',
    'Function__Student Transportation',
    'Function__Substitute Compensation',
    'Function__Teacher Compensation',
    'Function__Untracked Budget Set-Aside',
    'Function__Utilities',
    'Object_Type__Base Salary/Compensation',
    'Object_Type__Benefits',
    'Object_Type__Contracted Services',
    'Object_Type__Equipment & Equipment Lease',
    'Object_Type__NO_LABEL',
    'Object_Type__Other Compensation/Stipend',
    'Object_Type__Other Non-Compensation',
    'Object_Type__Rent/Utilities',
    'Object_Type__Substitute Compensation',
    'Object_Type__Supplies/Materials',
    'Object_Type__Travel & Conferences',
    'Operating_Status__Non-Operating',
    'Operating_Status__Operating, Not PreK-12',
    'Operating_Status__PreK-12 Operating',
    'Position_Type__(Exec) Director',
    'Position_Type__Area Officers',
    'Position_Type__Club Advisor/Coach',
    'Position_Type__Coordinator/Manager',
    'Position_Type__Custodian',
    'Position_Type__Guidance Counselor',
    'Position_Type__Instructional Coach',
    'Position_Type__Librarian',
    'Position_Type__NO_LABEL',
    'Position_Type__Non-Position',
    'Position_Type__Nurse',
    'Position_Type__Nurse Aide',
    'Position_Type__Occupational Therapist',
    'Position_Type__Other',
    'Position_Type__Physical Therapist',
    'Position_Type__Principal',
    'Position_Type__Psychologist',
    'Position_Type__School Monitor/Security',
    'Position_Type__Sec/Clerk/Other Admin',
    'Position_Type__Social Worker',
    'Position_Type__Speech Therapist',
    'Position_Type__Substitute',
    'Position_Type__TA',
    'Position_Type__Teacher',
    'Position_Type__Vice Principal',
    'Pre_K__NO_LABEL',
    'Pre_K__Non PreK',
    'Pre_K__PreK',
    'Reporting__NO_LABEL',
    'Reporting__Non-School',
    'Reporting__School',
    'Sharing__Leadership & Management',
    'Sharing__NO_LABEL',
    'Sharing__School Reported',
    'Sharing__School on Central Budgets',
    'Sharing__Shared Services',
    'Student_Type__Alternative',
    'Student_Type__At Risk',
    'Student_Type__ELL',
    'Student_Type__Gifted',
    'Student_Type__NO_LABEL',
    'Student_Type__Poverty',
    'Student_Type__PreK',
    'Student_Type__Special Education',
    'Student_Type__Unspecified',
    'Use__Business Services',
    'Use__ISPD',
    'Use__Instruction',
    'Use__Leadership',
    'Use__NO_LABEL',
    'Use__O&M',
    'Use__Pupil Services & Enrichment',
    'Use__Untracked Budget Set-Aside',    
]

# Benchmark: most common sub label

In [14]:
train_df = pd.read_csv(training_data_fp)
train_df.rename(columns={'Unnamed: 0':'_id'}, inplace=True)
train_df.head()

,_id,Function,Use,Sharing,Reporting,Student_Type,Position_Type,Object_Type,Pre_K,Operating_Status,...,Sub_Object_Description,Location_Description,FTE,Function_Description,Facility_or_Department,Position_Extra,Total,Program_Description,Fund_Description,Text_1
0,134338,Teacher Compensation,Instruction,School Reported,School,NO_LABEL,Teacher,NO_LABEL,NO_LABEL,PreK-12 Operating,...,NaN,NaN,1.0,NaN,NaN,KINDERGARTEN,50471.810,KINDERGARTEN,General Fund,NaN
1,206341,NO_LABEL,NO_LABEL,NO_LABEL,NO_LABEL,NO_LABEL,NO_LABEL,NO_LABEL,NO_LABEL,Non-Operating,...,NaN,NaN,NaN,RGN GOB,NaN,UNDESIGNATED,3477.860,BUILDING IMPROVEMENT SERVICES,NaN,BUILDING IMPROVEMENT SERVICES
2,326408,Teacher Compensation,Instruction,School Reported,School,Unspecified,Teacher,Base Salary/Compensation,Non PreK,PreK-12 Operating,...,NaN,NaN,1.0,NaN,NaN,TEACHER,62237.130,Instruction - Regular,General Purpose School,NaN
3,364634,Substitute Compensation,Instruction,School Reported,School,Unspecified,Substitute,Benefits,NO_LABEL,PreK-12 Operating,...,NaN,NaN,NaN,UNALLOC BUDGETS/SCHOOLS,NaN,PROFESSIONAL-INSTRUCTIONAL,22.300,GENERAL MIDDLE/JUNIOR HIGH SCH,NaN,REGULAR INSTRUCTION
4,47683,Substitute Compensation,Instruction,School Reported,School,Unspecified,Teacher,Substitute Compensation,NO_LABEL,PreK-12 Operating,...,NaN,NaN,NaN,NON-PROJECT,NaN,PROFESSIONAL-INSTRUCTIONAL,54.166,GENERAL HIGH SCHOOL EDUCATION,NaN,REGULAR INSTRUCTION


## Frequency proportion of sub labels

In [ ]:
train_df['Function'].value_counts(normalize=True)

In [27]:
print(TOP_LEVEL_LABELS)

['Function', 'Object_Type', 'Operating_Status', 'Position_Type', 'Pre_K', 'Reporting', 'Sharing', 'Student_Type', 'Use']


In [24]:
label_to_proportions = {}
for label in TOP_LEVEL_LABELS:
    print(label)
    proportions = train_df[label].value_counts(normalize=True)
    label_to_proportions[label] = proportions

Function
Object_Type
Operating_Status
Position_Type
Pre_K
Reporting
Sharing
Student_Type
Use


In [26]:
label_to_proportions['Function']

Teacher Compensation                               0.215736
Substitute Compensation                            0.155430
NO_LABEL                                           0.148844
Aides Compensation                                 0.049611
Instructional Materials & Supplies                 0.049243
Facilities & Maintenance                           0.049009
Professional Development                           0.047722
Student Transportation                             0.035903
Food Services                                      0.035483
School Administration                              0.032615
Enrichment                                         0.026697
Extended Time & Tutoring                           0.020923
Curriculum Development                             0.018994
Physical Health & Services                         0.014812
Social & Emotional                                 0.012499
Library & Media                                    0.011297
Special Population Program Management & 

In [28]:
print(FLATTENED_LABELS[0:5])

['Function__Aides Compensation', 'Function__Career & Academic Counseling', 'Function__Communications', 'Function__Curriculum Development', 'Function__Data Processing & Information Services']


## Create submission

In [32]:
flattened_scores = []

for flattened_label in FLATTENED_LABELS:
    top_label = flattened_label.split('__')[0]    
    sub_label = flattened_label.split('__')[1]    

    try:
        proportion = label_to_proportions[top_label][sub_label]
    except KeyError:
        proportion = 0
    
    flattened_scores.append(proportion)

In [68]:
print(flattened_scores[0:5])

[0.0496106446285947, 0.007222498419844258, 0.0007394879046260464, 0.018994346415107537, 0.008096892901665596]


In [36]:
sub_df = pd.read_csv(submission_format_fp)
sub_df.head()

,Unnamed: 0,Function__Aides Compensation,Function__Career & Academic Counseling,Function__Communications,Function__Curriculum Development,Function__Data Processing & Information Services,Function__Development & Fundraising,Function__Enrichment,Function__Extended Time & Tutoring,Function__Facilities & Maintenance,...,Student_Type__Special Education,Student_Type__Unspecified,Use__Business Services,Use__ISPD,Use__Instruction,Use__Leadership,Use__NO_LABEL,Use__O&M,Use__Pupil Services & Enrichment,Use__Untracked Budget Set-Aside
0,180042,0.027027,0.027027,0.027027,0.027027,0.027027,0.027027,0.027027,0.027027,0.027027,...,0.111111,0.111111,0.125,0.125,0.125,0.125,0.125,0.125,0.125,0.125
1,28872,0.027027,0.027027,0.027027,0.027027,0.027027,0.027027,0.027027,0.027027,0.027027,...,0.111111,0.111111,0.125,0.125,0.125,0.125,0.125,0.125,0.125,0.125
2,186915,0.027027,0.027027,0.027027,0.027027,0.027027,0.027027,0.027027,0.027027,0.027027,...,0.111111,0.111111,0.125,0.125,0.125,0.125,0.125,0.125,0.125,0.125
3,412396,0.027027,0.027027,0.027027,0.027027,0.027027,0.027027,0.027027,0.027027,0.027027,...,0.111111,0.111111,0.125,0.125,0.125,0.125,0.125,0.125,0.125,0.125
4,427740,0.027027,0.027027,0.027027,0.027027,0.027027,0.027027,0.027027,0.027027,0.027027,...,0.111111,0.111111,0.125,0.125,0.125,0.125,0.125,0.125,0.125,0.125


In [ ]:
# convoluted way because I could not figure how to set all row values...
for i, flattened_label in enumerate(FLATTENED_LABELS):
    print(i,flattened_label)
    sub_df.iloc[:][flattened_label] = flattened_scores[i]

In [63]:
sub_df.head()

,Unnamed: 0,Function__Aides Compensation,Function__Career & Academic Counseling,Function__Communications,Function__Curriculum Development,Function__Data Processing & Information Services,Function__Development & Fundraising,Function__Enrichment,Function__Extended Time & Tutoring,Function__Facilities & Maintenance,...,Student_Type__Special Education,Student_Type__Unspecified,Use__Business Services,Use__ISPD,Use__Instruction,Use__Leadership,Use__NO_LABEL,Use__O&M,Use__Pupil Services & Enrichment,Use__Untracked Budget Set-Aside
0,180042,0.049611,0.007222,0.000739,0.018994,0.008097,0.000232,0.026697,0.020923,0.049009,...,0.104987,0.557179,0.015289,0.06525,0.508668,0.03926,0.196644,0.114591,0.059406,0.000892
1,28872,0.049611,0.007222,0.000739,0.018994,0.008097,0.000232,0.026697,0.020923,0.049009,...,0.104987,0.557179,0.015289,0.06525,0.508668,0.03926,0.196644,0.114591,0.059406,0.000892
2,186915,0.049611,0.007222,0.000739,0.018994,0.008097,0.000232,0.026697,0.020923,0.049009,...,0.104987,0.557179,0.015289,0.06525,0.508668,0.03926,0.196644,0.114591,0.059406,0.000892
3,412396,0.049611,0.007222,0.000739,0.018994,0.008097,0.000232,0.026697,0.020923,0.049009,...,0.104987,0.557179,0.015289,0.06525,0.508668,0.03926,0.196644,0.114591,0.059406,0.000892
4,427740,0.049611,0.007222,0.000739,0.018994,0.008097,0.000232,0.026697,0.020923,0.049009,...,0.104987,0.557179,0.015289,0.06525,0.508668,0.03926,0.196644,0.114591,0.059406,0.000892


In [66]:
sub_df.to_csv("submission.csv",index=False)

In [67]:
!head ./submission.csv

Unnamed: 0,Function__Aides Compensation,Function__Career & Academic Counseling,Function__Communications,Function__Curriculum Development,Function__Data Processing & Information Services,Function__Development & Fundraising,Function__Enrichment,Function__Extended Time & Tutoring,Function__Facilities & Maintenance,Function__Facilities Planning,"Function__Finance, Budget, Purchasing & Distribution",Function__Food Services,Function__Governance,Function__Human Resources,Function__Instructional Materials & Supplies,Function__Insurance,Function__Legal,Function__Library & Media,Function__NO_LABEL,Function__Other Compensation,Function__Other Non-Compensation,Function__Parent & Community Relations,Function__Physical Health & Services,Function__Professional Development,Function__Recruitment,Function__Research & Accountability,Function__School Administration,Function__School Supervision,Function__Security & Safety,Function__Social & Emotional,Function__Special Population Program Management & Suppor